# Scalable Sustainability

We'll use the Planetary Computer's hub managed [Dask Gateway](https://gateway.dask.org/) to quickly spin up a Dask cluster.

In [ ]:
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
cluster.scale(64)
client = cluster.get_client()

cluster

## Cloud-free composite

This example will create a cloud-free composite of some Sentinel-2 Level 2-A scenes over Redmond, Washington. The Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1) helps us find just the scenes we need. Xarray, Dask, and NumPy will do the computation to remove the clouds.

In [ ]:
import rasterio.features
import pystac_client

bbox = [-122.28, 47.55, -121.96, 47.75]

stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

search = stac.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2016-01-01/2020-12-31",
    query={"eo:cloud_cover": {"lt": 25}},
)

items = list(search.get_items())
print(len(items))

The metadata at https://planetarycomputer.microsoft.com/api/stac/v1 is completely public and anonymously accessible.

The Planetary Computer's *data* files, the actual COGS, are typically in private blob storage accounts. But they can be accessed anonymously by signing the items.

In [ ]:
import planetary_computer

signed_items = [
    planetary_computer.sign(item).to_dict() for item in items
]

We'll use `stackstac` to quickly go from STAC items to a DataArray.

In [ ]:
import stackstac

data = (
    stackstac.stack(
        signed_items,
        assets=["B04", "B03", "B02"],  # red, green, blue
        chunksize=4096,
        resolution=30,
    )
    .where(lambda x: x > 0)  # sentinel-2 uses 0 as nodata
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)
data = data.persist()
data

We'll remove clouds by taking the median over time. We have a timeseries of points at each pixel. Since clouds come and go, the median pixel is not likely to be cloudy.

In [ ]:
median = data.median(dim="time")
median

In [ ]:
import xrspatial.multispectral

image = xrspatial.multispectral.true_color(
    *median
)
image = image.chunk({"x": 710, "y": 710}).persist()
image

stackstac has a handy `show` method to help visualize data that's on a cluster.

In [ ]:
from ipyleaflet import FullScreenControl

m = stackstac.show(
    image.assign_coords(epsg=data.coords["epsg"]).isel(band=slice(3)),
    range=[40, 255]
)
m.scroll_wheel_zoom = True
m.add_control(FullScreenControl())
m

That was nice, but we have all the power of a mature, domain-agnostic library like xarray! For example, suppose you don't want to combine images from January with images from July (you want to see snowy areas, for example). xarary provides a nice, high-level API to do just that:

In [ ]:
monthly = data.groupby("time.month").median().compute()

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
images = [
    xrspatial.multispectral.true_color(*x) for x in monthly
]
images = xr.concat(images, dim="time")

In [ ]:
g = images.plot.imshow(
    x="x", y="y", rgb="band", col="time", col_wrap=3, figsize=(12, 16)
)
for ax in g.axes.flat:
    ax.set_axis_off()

plt.tight_layout()

# Animatring Hurricane Florence

This example builds off an example from the [pytroll documentation](https://github.com/pytroll/pytroll-examples/blob/main/satpy/GOES-16%20ABI%20-%20True%20Color%20Animation%20-%20Hurricane%20Florence.ipynb). We'll make a short animation of Hurricane Florence as it moves across the Atlantic Ocean, using the [GOES-R Cloud & Moisture Imagery](https://planetarycomputer.microsoft.com/dataset/goes-cmi) product.

In [ ]:
import urllib.request

import contextily as ctx
import geopandas
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import planetary_computer
import pystac_client
import rioxarray
import xarray as xr

NOAA maintains a database of "storm tracks" which show the path that hurricanes and tropical storms take. We'll use it to find the position of Hurricane Florence.

In [ ]:
file, _ = urllib.request.urlretrieve(
    "https://www.ncei.noaa.gov/data/international-best-track-archive-for-"
    "climate-stewardship-ibtracs/v04r00/access/netcdf/IBTrACS.NA.v04r00.nc"
)
# The storm id comes from the text file in
# https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs
# /v04r00/access/netcdf/
# The name of this file changes with the update date, so we can't access it programatically.
STORM_ID = b"2018242N13343"
ds = xr.open_dataset(file)
storm_loc = (ds.sid == STORM_ID).argmax().item()

florence_data = ds.sel(storm=storm_loc)
florence_data

In [ ]:
df = (
    geopandas.GeoDataFrame(
        dict(
            time=pd.to_datetime(florence_data.time).tz_localize("UTC"),
            geometry=geopandas.points_from_xy(florence_data.lon, florence_data.lat),
        )
    )
    .set_crs(4326)
    .dropna()
)

start = pd.Timestamp("2018-09-11T13:00:00Z")
stop = pd.Timestamp("2018-09-11T15:40:00Z")

In [ ]:
ax = df.to_crs(epsg=3857).plot(figsize=(12, 12))
subset = df[df.time.dt.date == start.date()]
subset.to_crs(epsg=3857).plot(ax=ax, color="r")

ctx.add_basemap(ax)
ax.set_axis_off()
ax.set(title="Path of Hurricane Florence (animation period in red)");

In [ ]:
bbox = list(subset.total_bounds)
bbox

With those timestamps and bounding-box, we can do our search.

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/"
)
search = catalog.search(
    collections=["goes-cmi"],
    bbox=bbox,
    datetime=[start, stop],
    query={"goes:image-type": {"eq": "MESOSCALE"}},
)
items = search.get_all_items()
signed_items = sorted(
    [planetary_computer.sign(item) for item in items], key=lambda x: x.datetime
)

GOES doesn't have an epsg code, so it can't be read with `stackstac`. But `rioxarray` is a very flexible library for reading this raster data.

In [ ]:
ds = rioxarray.open_rasterio(signed_items[0].assets["C01_2km"].href).load()
ds[0].plot.imshow(figsize=(16, 9), cmap="Blues");

In [ ]:
bands = ["C01_2km", "C02_2km", "C03_2km"]
common_names = [
    items[0].assets[band].extra_fields["eo:bands"][0]["common_name"] for band in bands
]
time = xr.DataArray(
    pd.to_datetime([x.datetime for x in signed_items]).tz_localize(None),
    name="time",
    dims=["time"],
)

item = signed_items[0]
arrays = [rioxarray.open_rasterio(item.assets[band].href, chunks=True) for band in bands]

In [ ]:
%%time
arrays = [
    xr.concat(
        [rioxarray.open_rasterio(item.assets[band].href, chunks=True)
         for band in bands], dim="band"
    ).assign_coords(band=common_names)
    for item in signed_items
]
data = xr.concat(arrays, dim=time).rename("goes")
data

In [ ]:
data = data.persist()

GOES doesn't have a green band, so we simulate it.

In [ ]:
green = (
    0.45 * data.sel(band="red")
    + 0.1 * data.sel(band="nir09")
    + 0.45 * data.sel(band="blue")
).assign_coords(band="green")
green

And we apply a gamma-correction to make the picture prettier.

In [ ]:
γ = 2.2

rgb = xr.concat([data, green], dim="band").sel(band=["red", "green", "blue"])
rgb = rgb / rgb.max(dim=["band", "y", "x"])
rgb = rgb ** (1 / γ)
rgb = rgb.where(lambda x: x > 0, 0).where(lambda x: x < 1, 1)

In [ ]:
rgb = rgb.compute()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
rgb.isel(time=0).plot.imshow(rgb="band", add_labels=False)
ax.set_axis_off()

Now we animate this stack of images.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
ax.set_axis_off()

img = rgb[0].plot.imshow(ax=ax, add_colorbar=False, rgb="band", add_labels=False)
label = ax.text(
    0.4,
    0.03,
    pd.Timestamp(rgb.time.data[0]).isoformat(),
    transform=ax.transAxes,
    color="k",
    size=20,
)

def animate(i):
    img.set_data(rgb[i].transpose("y", "x", "band"))
    label.set_text(pd.Timestamp(rgb.time.data[i]).isoformat())
    return img, label


ani = animation.FuncAnimation(fig, animate, frames=len(rgb), interval=120)
ani.save(
    "goes.mp4",
    fps=15,
    extra_args=["-vcodec", "libx264"],
    savefig_kwargs=dict(pad_inches=0, transparent=True),
)

In [ ]:
from IPython.display import Video

Video("goes.mp4")

A pre-rendered version: https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_video/pc-examples-goes-florence.webm.

A couple things to highlight:

1. Once we had the date and times in geopandas, finding the right STAC items was pleasent

```python
search = catalog.search(
    collections=["goes-cmi"],
    bbox=bbox,
    datetime=[start, stop],
    query={"goes:image-type": {"eq": "MESOSCALE"}},
)
```

2. xarray's named axes are great! It's much nicer to use `"red"`, `"nir"` than `0`, `1`.